As seein in *'2. introduction to zarr'* notebook, zarr file have encodings called 'chunk'.  
 
How your data is 'chunk'ed plays important role on your computation time, through your cpu-time, and io-wait time.

1. If your chunk size is too small, it will use too many meta-data access thus your IO access gets slow. (and you might even slow down IO access of your colleagues,,  )
2. If your chunk order/size is not suitable to your computation,  each dask workers need to exchange data by communication, thus your computational time gets slow.  

For example, you have a 3D data in $t$, $i,$ and $j$ coordinate $ f (t,i,j)$ then, you would like to make $f'(i,j)$ as
$$
f'(i,j) = \sum_{t=tmin }^{t=tmax}f(t,i,j) .
$$

Lets say you have several Tera  bytes of data.  Clearly that wouldn't fits to your one dask worker's RAM size.  You'll need to chunk them, and your dataset is divided into chunks as shown here.
$$
   f (t,i,j) =  \sum_{tc=tcmin}^{tcmax,} \sum_{ic=icmin}^{icmax,} \sum_{jc=jcmin}^{jcmax,} fc_{tc, ic, jc} (t,i,j) \quad \\
$$
Thus, when dask compute $f'(i,j) $, it does
$$
f'(i,j) = \sum_{t=tmin }^{t=tmax}\sum_{tc=tcmin}^{tcmax,} \sum_{ic=icmin}^{icmax,} \sum_{jc=jcmin}^{jcmax,} fc_{tc, ic, jc} (t,i,j) \quad \\
$$

Here if $t$ is chunked as $1$ ,   $i$ and $j$are chunked as $ imax$ and $jmax$, we can simplyfy as
$$
f'(i,j) = \sum_{t=tmin }^{t=tmax}\sum_{ct=tmin}^{tmax,} fc_{ct } (t,i,j) \quad \\
$$

i.e., to have a specific value for $i$ and $j$ coordinate, your dask workers need to access $tmax$ number of $fc_{ct}$ files from your disk.

Here if $t$ is chunked as $tmax$ and  $ i$ and $ j$ are chunked as $imax$ and $jmax$ , we can simplify as 
$$
f'(i,j) = \sum_{t=tmin }^{t=tmax} \sum_{ic=icmin}^{icmax,} \sum_{jc=jcmin}^{jcmax,} fc_{ ic, jc} (t,i,j) \quad \\
$$

i.e., to have a specific value for $i$ and $j$ coordinate, your dask workers need to only access one file, $fc_{ic,jc}$ which have all time value needed.  

Above was an example when you want to integrate on time coordinate. But if you would like to integrate in i, or j coordinate, and would like to have $f'(t)$, other way of chunking would be suitable. 

So, to conclude, re-chunking plays important preparation phase before you starts to play with your Tera octes of datas
( Well until the day that re-chunking gets automatically done by dask??)


*** In this notebook, Lets see what one should take care when you want to rechunk your data.   ***

# 1. setting up enviroments.

In [1]:
import dask
import xarray as xr

In [2]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster(cores=24,memory='120 gb', walltime='1:00:00')
w = cluster.scale(30)

In [3]:
from dask.distributed import Client
client=Client(cluster)
client

Client Scheduler: tcp://10.120.40.174:41657 Dashboard: http://10.120.40.174:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


# 2. load data from zarr file as 'xarray data set' 

In [4]:
filename='/work/ALT/swot/swotpub/LLC4320/zarr/SST.zarr'
ds =xr.open_zarr(filename)

In [5]:
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, j: 4320, time: 8785)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<shape=(8785,), chunksize=(8785,)>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<shape=(8785,), chunksize=(1,)>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 3.732e+07 3.732e+07
Data variables:
    SST      (time, face, j, i) float32 dask.array<shape=(8785, 13, 4320, 4320), chunksize=(1, 1, 4320, 4320)>

 data size: 8525.4 GB



# 3. Lets now try some other way of chunking.  
For example one may want to have time integral of small region.  As explained above, to do that, chunk should be organised in the way so that the same chunk contains all the time seriese of one geographical coordinate. 

Example below let dask to chose automatically the size of chunk for dimentions face, j and i, according to the other constraint (i.e. time does not get chunked as it is specified as -1)  and dask's default chunk size which is 128MiB

In [6]:
dask.config.get('array.chunk-size')

'128MiB'

In [7]:
dstmp =  ds.chunk({'time': -1,'face':'auto', 'j':'auto', 'i':'auto'})
print(dstmp)
print('\n data size: %.1f GB' %(dstmp.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, j: 4320, time: 8785)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<shape=(8785,), chunksize=(8785,)>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<shape=(8785,), chunksize=(8785,)>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 3.732e+07 3.732e+07
Data variables:
    SST      (time, face, j, i) float32 dask.array<shape=(8785, 13, 4320, 4320), chunksize=(8785, 1, 60, 60)>

 data size: 8525.4 GB


dask chose your chunk size as '8785 for time, 1 for face, 60 for i and j.  
Here, chunk , is the shape of your chunk.  So when one write this file in to a zarr file, you'll find each zarr chunk file contains, 8785 time cordinate, 1 face value and 60 for each i and j, thus we should find
$$
\frac{8785}{8785}  \times \frac{13}{1}  \times \frac{4320}{60}  \times \frac{4320}{ 60} = 63292
$$
so we will have $63292$ chunked files after dumping them to a zarr .  
ATT. before the transformation, there were $114205$ chunked files

In [8]:
%time dstmp.to_zarr('/work/scratch/odakat/dstmp.zarr',  mode='w')

ValueError: Final chunk of Zarr array must be the same size or smaller than the first. The specified Zarr chunk encoding is (1, 1, 4320, 4320), but (8785,) in variable Dask chunks ((8785,), (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60), (60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60)) is incompatible. Consider rechunking using `chunk()`.

# 4. It does not work. why? 
1.  Attention a small trick bug here.  We re-define the chunk, but the chunk encoding does not re-defined automatically.  Thus, you'll need to delete those encodings as shown below.  

2. my first try was to rechunk all the data at once, hopeing that dask can automatically parallelize the re-chunk process,,, But, it does not work.  we transform here $114205$ chunked files to $63292$ chunked file.  Probably we asked too much work at once to dask.  (we will see more in dtail on step 6)


# 5.  Clean up the encoding chunk to avoid the bug.
- You may think that one just need to create a 'zarr' file without chunk encoding: I have tried that (as shown in 3.1 zarr's chunk encoding) for the moemnt I did not suceed to get rid of cleaning encoding chunks 


In [4]:
filename='/work/ALT/swot/swotpub/LLC4320/zarr/SST.zarr'
ds =xr.open_zarr(filename)
del ds.SST.encoding['chunks']
del ds.iters.encoding['chunks']
del ds.dtime.encoding['chunks']
print(ds.SST.encoding)
print(ds.iters.encoding)
print(ds.dtime.encoding)

{'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0), 'filters': None, '_FillValue': nan, 'dtype': dtype('float32'), 'coordinates': 'dtime iters'}
{'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0), 'filters': None, 'dtype': dtype('int64')}
{'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0), 'filters': None, 'units': 'hours since 2011-11-15 00:00:00', 'calendar': 'proleptic_gregorian', 'dtype': dtype('int64')}


# 6.  try again 

To create face of new format chunked file, dask client need to access to $8785$ chunked files for each of $\frac{4320}{60} \ times \frac{4320}{60}$ which makes $45541440  $ for each face.  
We have 13 face here, thus it will create 600 M file access.  Total file size of ds is 8525.4GB.

## 6.1
Would xarray/dask capable of handling that when we give them 3To of RAM?

In [10]:
ds =  ds.chunk({'time': -1,'face':'auto', 'j':'auto', 'i':'auto'})
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, j: 4320, time: 8785)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<shape=(8785,), chunksize=(8785,)>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<shape=(8785,), chunksize=(8785,)>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 3.732e+07 3.732e+07
Data variables:
    SST      (time, face, j, i) float32 dask.array<shape=(8785, 13, 4320, 4320), chunksize=(8785, 1, 60, 60)>

 data size: 8525.4 GB


In [6]:
w = cluster.scale(30)
client

Client Scheduler: tcp://10.120.40.174:40829 Dashboard: http://10.120.40.174:8787/status,Cluster Workers: 9 Cores: 216 Memory: 1.08 TB


In [11]:
print(ds.SST.encoding)
print(ds.iters.encoding)
print(ds.dtime.encoding)

{'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0), 'filters': None, '_FillValue': nan, 'dtype': dtype('float32'), 'coordinates': 'dtime iters'}
{'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0), 'filters': None, 'dtype': dtype('int64')}
{'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0), 'filters': None, 'units': 'hours since 2011-11-15 00:00:00', 'calendar': 'proleptic_gregorian', 'dtype': dtype('int64')}


In [20]:
filename='/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr'
%time  ds.to_zarr(filename,  mode='w')

distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)


KeyboardInterrupt: 

Nope.  even after 30 min, nothing happens. Thus we take another try.

## 6-2 split the computation in each face.

We can make a tmp file for each face, then concartenate them in the end. 
But what would be the best size to deal with it?
(8585,0,60,60) correspoinds to 128M of data size.  
i.e. for each face, 128M x 72 x 72 = 663G of data size.  
The number of computational step would go down to 45M.
Total memory size of dask woker are 3T as **6-1**


In [ ]:
F = range(13)  
filename = '/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr'
for face in F:
    %time dstmp =  ds.isel(face=face).chunk({'time': -1,'j':60, 'i':60})
    tmp = filename+'.'+str(face)+'.zarr'
    print(tmp)
    print('\n data size: %.1f GB' %(dstmp.nbytes / 1e9))
    %time dstmp.to_zarr(tmp,  mode='w')
print(filename)

CPU times: user 2.83 s, sys: 267 ms, total: 3.1 s
Wall time: 3.08 s
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.0.zarr

 data size: 655.8 GB
CPU times: user 9min 20s, sys: 33.4 s, total: 9min 54s
Wall time: 19min 20s
CPU times: user 4 s, sys: 200 ms, total: 4.2 s
Wall time: 4.17 s
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.1.zarr

 data size: 655.8 GB
CPU times: user 9min 12s, sys: 31.5 s, total: 9min 44s
Wall time: 18min 48s
CPU times: user 6.03 s, sys: 89.5 ms, total: 6.12 s
Wall time: 6.07 s
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.2.zarr

 data size: 655.8 GB


KeyboardInterrupt: 

CPU times: user 3.96 s, sys: 52.6 ms, total: 4.01 s
Wall time: 3.98 s
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.3.zarr

 data size: 655.8 GB


KeyboardInterrupt: 

CPU times: user 1.61 s, sys: 13.7 ms, total: 1.62 s
Wall time: 1.61 s
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.4.zarr

 data size: 655.8 GB


...

KeyboardInterrupt: 

/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.11.zarr

 data size: 655.8 GB


KeyboardInterrupt: 

Exception ignored in: <bound method GCDiagnosis._gc_callback of <distributed.utils_perf.GCDiagnosis object at 0x2b53604e5d68>>
Traceback (most recent call last):
  File "/home/mp/odakat/miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils_perf.py", line 178, in _gc_callback
    def _gc_callback(self, phase, info):
KeyboardInterrupt
Exception ignored in: <bound method GCDiagnosis._gc_callback of <distributed.utils_perf.GCDiagnosis object at 0x2b53604e5d68>>
Traceback (most recent call last):
  File "/home/mp/odakat/miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils_perf.py", line 188, in _gc_callback
    self._fractional_timer.start_timing()
  File "/home/mp/odakat/miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils_perf.py", line 111, in start_timing
    assert self._cur_start is None
AssertionError: 


CPU times: user 3.43 s, sys: 25.7 ms, total: 3.45 s
Wall time: 3.42 s
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.12.zarr

 data size: 655.8 GB


 1/13 of creating file took 20 min.  It feels too long, but the computation is not stuck. 
Diagnostic says total task is 388 K.
Max memory used by dask workers was 1.2T (about twice the size of final file size.)

## 6-3 split the computation time more. 


We can make a concatenation for each face.  
But what would be the best size to deal with it?
(8585,0,60,60) correspoinds to 128M of data size.  
by splitting the file in J order, 4320/60=72 task can be created, and each of file size goes down to
128M * 13* 72 = 120G.


In [7]:
client

Client Scheduler: tcp://10.120.40.174:40829 Dashboard: http://10.120.40.174:8787/status,Cluster Workers: 11 Cores: 264 Memory: 1.32 TB


In [ ]:
bj=int(4320/60)
print(bj)
J = range(bj)
filename = '/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr'
for nbj in J:
    jmin=nbj*60
    jmax=(nbj+1)*60
    %time dstmp =  ds.isel(j=slice(jmin,jmax)).chunk({'time': -1,'face':1, 'i':60})
    tmp = filename+'.j'+str(nbj)+'.zarr'
    print(tmp)
    print('\n data size: %.1f GB' %(dstmp.nbytes / 1e9))
    %time dstmp.to_zarr(tmp,  mode='w')


72
CPU times: user 1.49 s, sys: 176 ms, total: 1.66 s
Wall time: 1.64 s
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.j0.zarr

 data size: 118.4 GB
CPU times: user 8min 1s, sys: 24.5 s, total: 8min 25s
Wall time: 15min 21s
CPU times: user 3.04 s, sys: 244 ms, total: 3.29 s
Wall time: 3.26 s
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.j1.zarr

 data size: 118.4 GB


KeyboardInterrupt: 

CPU times: user 2.45 s, sys: 69.6 ms, total: 2.52 s
Wall time: 2.49 s
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.j2.zarr

 data size: 118.4 GB


 ~ with bug with 30 dask worker still 1/72 of creating file took 12 min. file size decreased 1/5, but computational time did not go down even half. Diagnostic says total task is 295k  ~

with 11 dask worker took 15 min, diagnostic says total task is 304k, max memory stayed around 70G. 

one more try.

## 6-4 split more




In [6]:
client

Client Scheduler: tcp://10.120.40.174:41657 Dashboard: http://10.120.40.174:8787/status,Cluster Workers: 20 Cores: 480 Memory: 2.40 TB


In [ ]:
F = range(13)  
bj=int(4320/60)
J = range(bj)
filename = '/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr'
for face in F:
    for nbj in J:
        jmin=nbj*60
        jmax=(nbj+1)*60
        %time dstmp =  ds.isel(face=face,j=slice(jmin,jmax)).chunk({'time': -1,'j':60, 'i':60})
        tmp = filename+'.f'+str(face)+'.j'+str(nbj)+'.zarr'
        print(tmp)
        print('\n data size: %.1f GB' %(dstmp.nbytes / 1e9))
        %time dstmp.to_zarr(tmp,  mode='w')
filename = '/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr'


CPU times: user 146 ms, sys: 1.03 ms, total: 147 ms
Wall time: 143 ms
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.f0.j0.zarr

 data size: 9.1 GB


distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)


CPU times: user 33.8 s, sys: 1.78 s, total: 35.5 s
Wall time: 58.7 s
CPU times: user 146 ms, sys: 1.02 ms, total: 147 ms
Wall time: 145 ms
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.f0.j1.zarr

 data size: 9.1 GB


distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)


CPU times: user 37.4 s, sys: 1.85 s, total: 39.3 s
Wall time: 1min 4s
CPU times: user 148 ms, sys: 4.06 ms, total: 152 ms
Wall time: 148 ms
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.f0.j2.zarr

 data size: 9.1 GB


...

CPU times: user 37.4 s, sys: 1.86 s, total: 39.2 s
Wall time: 59.6 s
CPU times: user 152 ms, sys: 1.01 ms, total: 153 ms
Wall time: 150 ms
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.f0.j7.zarr

 data size: 9.1 GB


distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


CPU times: user 38.6 s, sys: 2.12 s, total: 40.7 s
Wall time: 56.2 s
CPU times: user 156 ms, sys: 9.12 ms, total: 165 ms
Wall time: 162 ms
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.f0.j8.zarr

 data size: 9.1 GB


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


CPU times: user 42.9 s, sys: 2.32 s, total: 45.3 s
Wall time: 1min 25s
CPU times: user 150 ms, sys: 1.06 ms, total: 151 ms
Wall time: 148 ms
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.f0.j9.zarr

 data size: 9.1 GB
CPU times: user 36.6 s, sys: 1.89 s, total: 38.5 s
Wall time: 55.7 s
CPU times: user 153 ms, sys: 1.01 ms, total: 154 ms
Wall time: 152 ms
/work/ALT/swot/swotpub/LLC4320/zarr/chunktime/SST.zarr.f0.j10.zarr

 data size: 9.1 GB


with 20 dask worker it took 1 min , diagnostic says total task is 31k
 and compare to ***6-2***, the computational task is 1/72, and time is 1/20.


## 7 conclusion. 

we need to verify the computational time with dask 30 worker, but splitting the file in to face=13 looks fastest.  Estimation time is  $20min \times 13 + concartenation \quad time $.  Which will not fit in the normal 'quick' queue i have access too, thus we'll finalise this using jupyter note book submission (notebook 4) 

*** Tips.  How do we chose the right size of chunking and right way to do re-chunking?? ***
- 1. findout final chunk size using 'auto' function 
- 2. if your array for re-chunking is huge,  use a tmp dask array and concartenate.
- 3. Total size of dask -worker RAM should be at least twice the size of each of your dask tmp array, and use the max tmp array size as possible from your dask-cluster you have to go fast. 
- 4. Monitoring showed that one do not need numbers of core for executing re-chunking.  (will be verified in notebook 4)
- 5. If computation takes too long, submit your jupyternotebook to a queue ( notebook 4) 

In [ ]:
cluster.close()